# The goal of this notebook is to fix issues with duplicated reactions and other incositencies

In [17]:
%matplotlib inline
import os
import sys
sys.path.append('/home/sergio/Dropbox/s/cthermgem-dev')
import cobra as cb
import tools.ms2bigg

In [18]:
model_ms = cb.io.read_sbml_model(os.path.join(settings.PROJECT_ROOT,'iCBI','iCBI665_v3.sbml'))
model = tools.ms2bigg.main(model_ms)

# Consolidate reactions present in separately in both directions
I spotted the case of GRTT and FRTT, the same reaction  but in different directions, coded by the same gene. To avoid TICs this should be a reversible reaction.
Let's do the analysis in a systematic way


In [19]:
same_mets = []
for rxn_1 in model.reactions:
    met_ids_1 = [met.id for met in rxn_1.metabolites.keys()]
    for rxn_2 in model.reactions:
        met_ids_2 = [met.id for met in rxn_2.metabolites.keys()]
        if rxn_1.id != rxn_2.id:
            if set(met_ids_1) == set(met_ids_2):
                pair = {rxn_1.id, rxn_2.id}
                if pair not in same_mets:
                    same_mets.append(pair)
                    print(pair)


{'PGMT', 'PGCM'}
{'FRTT', 'GRTT'}
{'ATPM', 'NA1abc'}
{'ATPM', 'FUCabc'}
{'GLCabc', 'ATPM'}
{'SBTabc', 'ATPM'}
{'FRUabc', 'ATPM'}
{'FUCabc', 'NA1abc'}
{'GLCabc', 'NA1abc'}
{'SBTabc', 'NA1abc'}
{'FRUabc', 'NA1abc'}
{'GLCabc', 'FUCabc'}
{'SBTabc', 'FUCabc'}
{'FRUabc', 'FUCabc'}
{'GLCabc', 'SBTabc'}
{'FRUabc', 'GLCabc'}
{'FRUabc', 'SBTabc'}
{'BIOMASS_CELLULOSE', 'BIOMASS_CELLOBIOSE'}
{'BIOMASS_CELLULOSE', 'BIOMASS_NO_CELLULOSOME'}
{'BIOMASS_NO_CELLULOSOME', 'BIOMASS_CELLOBIOSE'}


- PGMT-PGMC and FRTT-GRTT seem to be actual cases of duplication.
- ATPM and abc transporter point at an issue with abc trasnporters lacking the metabolite they should translocate
- BIOMASS reactions are "false positives" since they are expected to have the same metabolties.

In [23]:
isg = cb.io.load_json_model(os.path.join(settings.PROJECT_ROOT,'iSG676','iSG676_cb.json'))

In [24]:
model.reactions.GLCabc.reaction

'atp_c + h2o_c --> adp_c + h_c + pi_c'

In [26]:
isg.reactions.GLCabc.reaction

'atp_c + glc__D_e + h2o_c --> adp_c + glc__D_c + h_c + pi_c'

The issue with sugar ABC transporters was introduced somwhere along the conversion from iSG to iCBI

# Fix real duplicates

In [20]:
model.reactions.PGMT

Reaction identifier,PGMT
Name,alpha_D_Glucose_1_phosphate_1_6_phosphomutase_c0
Memory address,0x07f534e3704a8
Stoichiometry,g1p_c <=> g6p_c Glucose_1_phosphate_c0 <=> D_glucose_6_phosphate_c0
GPR,CLO1313_RS05070
Lower bound,-1000.0
Upper bound,1000.0


In [22]:
model.reactions.PGCM

Reaction identifier,PGCM
Name,alpha_D_Glucose_1_phosphate_1_6_phosphomutase_c0
Memory address,0x07f534e5c49e8
Stoichiometry,g1p_c <=> g6p_c Glucose_1_phosphate_c0 <=> D_glucose_6_phosphate_c0
GPR,CLO1313_RS05070
Lower bound,-1000.0
Upper bound,1000.0


# Fix transporters

# Save updated model